In [1]:
# Storage

In [2]:
import logging
from utils import *
import smartpynector as sp

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [4]:
# Constants
READ_URL = "https://graphdb.odissei.nl/repositories/MateuszTest"
WRITE_URL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
PREFIXES={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    }

GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                     ?meas saref:hasValue ?temp .
                     ?meas saref:isMeasuredIn saref:TemperatureUnit .
                     ?meas saref:hasTimestamp ?timestamp .
                     ?meas saref:isMeasurementOf ?room_id .
                     ?meas saref:relatesToProperty saref:Temperature .
                     ?meas saref:measurementMadeBy ?device_id ."""


In [5]:
measurements = []

def handle_answer_measurements(query_bindings):
    global measurements
    print(f'Answer query bindings: {query_bindings}')
    sp.store_data_in_graphdb(
        graph_pattern=GRAPH_PATTERN,
        binding_set=query_bindings,
        prefixes=PREFIXES,
        read_url=READ_URL,
        write_url=WRITE_URL,
    )
    return match_bindings(query_bindings, measurements)


def handle_react_measurements(bindings):
    global measurements
    measurements += bindings
    # print(f'React bindings: {bindings}')
    sp.store_data_in_graphdb(
        graph_pattern=GRAPH_PATTERN,
        binding_set=bindings,
        prefixes=PREFIXES,
        read_url=READ_URL,
        write_url=WRITE_URL,
    )
    return []


In [6]:
def start_storage_kb(kb_id, kb_name, kb_description, ke_endpoint):
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    answer_measurements_ki = register_answer_knowledge_interaction(
        GRAPH_PATTERN,
        "answer-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    react_measurements_ki = register_react_knowledge_interaction(
        GRAPH_PATTERN,
        None,
        "react-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    start_handle_loop(
        {
            answer_measurements_ki: handle_answer_measurements,
            react_measurements_ki: handle_react_measurements,
        },
        kb_id,
        ke_endpoint,
    )



In [7]:
start_storage_kb(
    "http://example.org/storage",
    "Storage",
    "GraphDB smart connector",
    "http://knowledge_engine:8280/rest/",
)

INFO:utils:registered Storage
INFO:utils:received issued knowledge interaction id: http://example.org/storage/interaction/answer-measurements
INFO:utils:received issued knowledge interaction id: http://example.org/storage/interaction/react-measurements


KeyboardInterrupt: 